# Test notebook

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import folium
import streamlit as st
import os
import matplotlib
from rasterstats import zonal_stats
import rasterio
import requests

from src.config_parameters import params
from src.utils import *
from src.utils_proximity import *
from src.utils_population import *
from src.utils_plotting import *

## General parameters

In [3]:
verbose = True
use_default_data = True
use_local_pop_data = False

## User input

In [4]:
# Parameters given by the user
upload_poi_file = "test_data/mda_health_hospitals.zip"
upload_aoi_file = "test_data/mda_admn_ad0.zip" 
poi_name_col = "name"

## Process input data

In [5]:
# Create geopandas dataframes and prepare starting points
aoi_gdf = create_aoi_gpd(upload_aoi_file)
poi_gdf = create_poi_gpd(upload_poi_file)

poi_within_aoi, poi_outside_aoi = poi_v_aoi(aoi_gdf, poi_gdf)
start_points_dict, map_centre = prep_user_poi(poi_gdf, poi_name_col)

/home/daniele/Documents/MapAction/Python_projects/proximity-tool/app/src/utils_proximity.py:336: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  map_cent = poi_gdata.dissolve().to_crs("EPSG:4326").centroid


## Run computation

In [6]:
# Initialize ORS client
ors = ors_initialize()

In [7]:
# API call
all_isos, api_time = get_isochrones(start_points_dict, ors, mock_function=use_default_data)

2023-03-09 16:19:49.452 ERROR   fiona._env: app/test_data/mda_isochrones.geojson: No such file or directory


In [12]:
# Dissolve isochrones
diss_isoc, diss_time = dissolve_iso(all_isos)

In [16]:
# Remove overlaps from isochrones
diff_isoc, diff_time = difference_iso(diss_isoc)

In [17]:
# Fill to aoi area
diff_isoc, fill_time = fill_aoi(diff_isoc, diss_isoc, aoi_gdf)

In [18]:
# Add population analysis
diff_isoc = add_population_data(
    diff_isoc, 
    tif_folder='test_data/pop_data',
    use_local_tif=use_local_pop_data
)

/home/daniele/miniconda3/envs/proximity/lib/python3.9/site-packages/rasterstats/io.py:335: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


## Run computation in one function

In [ ]:
diff_isoc = run_analysis(
    poi_gdf=poi_gdf, 
    poi_name_col=poi_name_col,
    aoi_gdf=aoi_gdf,
    use_default_data=use_default_data,
    add_pop_data=True,
    pop_folder='test_data/pop_data',
    use_local_pop_data=use_local_pop_data,
    verbose=verbose,
    text_on_streamlit=False,
    )

## Plots

In [ ]:
m = plot_isochrones(
    diff_isoc,
    poi_gdf=poi_gdf, 
    poi_name_col=poi_name_col,
    add_legend=True
)
m

In [ ]:
fig = plot_population_summary(diff_isoc)